### エラーデータの欠損値削除とパラメータ化

In [697]:
!pip3 install kogi
from kogi.liberr.emodel import extract_params_from_error
import pandas as pd

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


##### 欠損値処理（output：error_train_new.csv, error_test_new.csv）

In [698]:
#trainデータの欠損値処理

df_train = pd.read_csv('error_train.tsv', sep='\t', header=None)

df_train_0 = df_train[0].str.split('<tab>', expand=True)
df_train_1 = df_train[1].str.split('<tab>', expand=True)

df_train_new = pd.concat([df_train_0, df_train_1], axis=1)
df_train_new = df_train_new.set_axis([0, 1, 2, 3], axis=1)
df_train_new_drop = df_train_new[(df_train_new[0]!='') & (df_train_new[1]!='') & (df_train_new[2]!='') & (df_train_new[3]!='') ]
list_0=[]
list_1=[]
for a,b,c,d in zip(df_train_new_drop[0], df_train_new_drop[1], df_train_new_drop[2], df_train_new_drop[3]):
    list_0.append(a+'<tab>'+b)
    list_1.append(c+'<tab>'+d)
df_train_new = pd.DataFrame(list(zip(list_0,list_1)), columns = [0,1])
df_train_new = df_train_new.drop(df_train_new.index[5]) #入力途中のものを目視で発見
df_train_new.to_csv('error_train_new.tsv', sep='\t', header=None, index=None)

In [699]:
#testデータの欠損値処理

df_test = pd.read_csv('error_test.tsv', sep='\t', header=None)

df_test_0 = df_test[0].str.split('<tab>', expand=True)
df_test_1 = df_test[1].str.split('<tab>', expand=True)

df_test_new = pd.concat([df_test_0, df_test_1], axis=1)
df_test_new = df_test_new.set_axis([0, 1, 2, 3], axis=1)
df_test_new_drop = df_test_new[(df_test_new[0]!='') & (df_test_new[1]!='') & (df_test_new[2]!='') & (df_test_new[3]!='') ]
list_0=[]
list_1=[]
for a,b,c,d in zip(df_test_new_drop[0], df_test_new_drop[1], df_test_new_drop[2], df_test_new_drop[3]):
    list_0.append(a+'<tab>'+b)
    list_1.append(c+'<tab>'+d)
df_test_new = pd.DataFrame(list(zip(list_0,list_1)), columns = [0,1])
df_test_new.to_csv('error_test_new.tsv', sep='\t', header=None, index=None)

##### パラメータ化（output：error_train_params.csv, error_test_params.csv）

In [700]:
def error_params(df, df0, df1, output:str):
    emsg_params_list=[]
    code_params_list=[]
    cause_params_list=[]
    solution_params_list=[]

    params = {i:chr(c) for i,c in enumerate(range(ord('A'),ord('Z')+1))}

    for emsg, code, cause, solution in zip(df0[0], df0[1], df1[0], df1[1]):
        emsg_params = extract_params_from_error(emsg)[0]
        extract_params=set()
        for i in extract_params_from_error(emsg)[1]:
            try:
                extract_params.add(''.join(eval(i.replace("'(",'').replace(")'",'')) ))
            except:
                extract_params.add(''.join(i.replace("'(",'').replace(")'",'')))
        if '' in extract_params:
            extract_params.remove('')
        extract_params = list(extract_params)
        if extract_params:
            for i in range(len(extract_params)):
                code_params = code.replace(extract_params[i], params[i])
                cause_params = cause.replace(extract_params[i], params[i])
                solution_params = solution.replace(extract_params[i], params[i])
        else:
            code_params = code
            cause_params = cause
            solution_params = solution
        
        emsg_params_list.append(emsg_params)
        code_params_list.append(code_params) 
        cause_params_list.append(cause_params) 
        solution_params_list.append(solution_params)

    df_0_params_list=[]
    df_1_params_list=[]

    for i in range(len(df)):
        df_0_params_list.append(emsg_params_list[i]+'<sep>'+code_params_list[i])
        df_1_params_list.append(cause_params_list[i]+'<sep>'+solution_params_list[i])

    df_params = pd.DataFrame(list(zip(df_0_params_list, df_1_params_list)))
    df_params.to_csv(output, sep='\t', header=None, index=None)

In [701]:
df_train_0 = df_train_new[0].str.split('<tab>', expand=True)
df_train_1 = df_train_new[1].str.split('<tab>', expand=True)

error_params(df_train_new, df_train_0, df_train_1, 'error_train_params.tsv')
error_params(df_test_new, df_test_0, df_test_1, 'error_test_params.tsv')

##### debug

In [702]:
def inputnum(i):
    return df_train_0[0][i], df_train_0[1][i], df_train_1[0][i], df_train_1[1][i]

def pri(i):
    print(emsg_params_list[i])
    print(code_params_list[i])
    print(cause_params_list[i])
    print(solution_params_list[i])

emsg, code, cause, solution = inputnum(257)
emsg_params = extract_params_from_error(emsg)[0]
extract_params=set()
for i in extract_params_from_error(emsg)[1]:
    try:
        extract_params.add(''.join(eval(i.replace("'(",'').replace(")'",'').strip(' ')) ))
    except:
        extract_params.add(''.join(i.replace("'(",'').replace(")'",'').strip(' ')))
if '' in extract_params:
        extract_params.remove('')
extract_params = list(extract_params)
for i in range(len(extract_params)):
    code_params = code.replace(extract_params[i], params[i])
    cause_params = cause.replace(extract_params[i], params[i])
    solution_params = solution.replace(extract_params[i], params[i])